##  Defining new types

In this notebook, we will look at how to define new types in Julia. This is very common when you need a certain **kind (type) of behaviour** that is not captured by the "built-in" types.

Note that types that you define are, in fact, on the same footing as the "built-in" types in the standard library (`Int`, `Float64`, etc.), which are themselves defined in Julia code.

## A wrapper type

We will look at a concrete example that arose naturally in real code in the author's pacakage 
[`IntervalConstraintProgramming.jl`](https://github.com/dpsanders/IntervalConstraintProgramming.jl): defining a type  to represent the volume of an object. This is similar to a [post on the `julia-users` mailing list](https://groups.google.com/forum/#!searchin/julia-users/probability/julia-users/PpXLxHajsfA/UWpmJZd2BQAJ) about defining a type to represent a probability.  

Despite the fact that the type contains only a single field, many of the issues that confuse people can be appreciated in this apparently simple situation.

We could choose to represent a volume with a standard `Float64`, but, as we shall see, there are good reasons to wrap this into a new type:

In [1]:
immutable Vol
    value::Float64
end

We can now create an object of type `Vol` with

In [5]:
methods(Vol)

4-element Array{Any,1}:
 call(::Type{Vol}, value::Float64) at In[1]:2   
 call(::Type{Vol}, value) at In[1]:2            
 call{T}(::Type{T}, arg) at essentials.jl:56    
 call{T}(::Type{T}, args...) at essentials.jl:57

In [3]:
V = Vol(3)

Vol(3.0)

In [4]:
W = Vol(10.5)

Vol(10.5)

In [6]:
Vol("David")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[6], in expression starting on line 1

In [8]:
convert(Float64, "David")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[8], in expression starting on line 1

In [9]:
convert(Float64, "3.5")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[9], in expression starting on line 1

In [10]:
parse(Float64, "3.5")

3.5

Let's recall how to look inside the object. We can do it interactively with `V.<TAB>` [press the `TAB` key after typing `V.` at the REPL, in the notebook, or in Atom].

Alternatively, we can use `fieldnames`, which returns an array of `Symbol`s, containing the names of the fields:

In [11]:
V

Vol(3.0)

In [13]:
V.value = 10

LoadError: LoadError: type Vol is immutable
while loading In[13], in expression starting on line 1

In [14]:
fieldnames(V)

1-element Array{Symbol,1}:
 :value

We access the field with any of the following commands:

In [15]:
V.value

3.0

In [16]:
getfield(V, :value)  # by name

3.0

In [17]:
getfield(V, 1)       # by number

3.0

If the type were declared with `type`, instead of `immutable`, then we could also change the value via `setfield`.
This gives an error for an immutable object:

In [ ]:
V.value = 4

## Constructors

So far, we have been using the **default constructors**. Recall that the constructors for a type are functions that create objects of that type. We can see which constructors are available using `methods`:

In [18]:
methods(Vol)

4-element Array{Any,1}:
 call(::Type{Vol}, value::Float64) at In[1]:2   
 call(::Type{Vol}, value) at In[1]:2            
 call{T}(::Type{T}, arg) at essentials.jl:56    
 call{T}(::Type{T}, args...) at essentials.jl:57

The output from Julia v0.5 is clearer:

    # 3 methods for generic function "(::Type)":
    Vol(value::Float64) at REPL[1]:2
    Vol(value) at REPL[1]:2
    (::Type{T}){T}(arg) at sysimg.jl:60

The problem is that we can do the following:

In [19]:
Vol(-1)

Vol(-1.0)

This should not be allowed, since volumes cannot be negative. We want to impose a constraint *at the moment when we  create the object*, i.e. in the constructor. To do so, we define an **inner constructor** inside the type definition.

Since Julia does not allow redefining types, we need to clear the workspace before we do so:

In [29]:
workspace()

In [32]:
isa(3, Real)

true

In [33]:
immutable Vol
    value::Float64
    
    function Vol(x)
        isa(x, Real) || throw(ArgumentError("Must be real"))
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new(x)
    end
end

In [34]:
Vol("hola")

LoadError: LoadError: ArgumentError: Must be real
while loading In[34], in expression starting on line 1

In [35]:
Vol(-3)

LoadError: LoadError: ArgumentError: Negative volume not allowed.
while loading In[35], in expression starting on line 1

Here we have used the so-called ["short-circuit" evaluation](http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-short-circuit-evaluation) operator `&&`, which can be thought of as an `if...then`.

The function `new` is a special function that actually instantiates the object. If the type has several fields, some may be left empty, by not including them in the call to `new`, and filled in later (if the type is not `immutable`).

In [ ]:
Vol(3)

In [ ]:
Vol(-1)

## Making types parametric

There is no reason why volumes should be floating-point numbers -- they may be any kind of number. We could just remove the type annotation:

In [36]:
workspace()

In [37]:
immutable Vol
    value   # DO NOT DO THIS!
end

In [38]:
Vol(3.5)

Vol(3.5)

In [39]:
Vol(3)

Vol(3)

In [41]:
Vol("hello")

Vol("hello")

but we should **never do this**.  [By "never", we of course mean "only if you are really sure that you know what you're doing" and won't complain about the consequences.] The reason for this is that Julia is unable to infer the type of the variable, and so will be **very slow**.

Instead, what we really mean is that there should be a *different type* of `Vol` for each type that it could contain - integers, floats, `BigFloat`s, rationals, intervals, etc.
To tell Julia this, we use a **type parameter**, written with curly braces:

In [42]:
workspace()

In [43]:
immutable Vol{T}
    value::T   # DO THIS INSTEAD!
end

In words, this says "for **any** type `T`, a `Vol` with type parameter `T` looks like the following". This makes `Vol` a **parametric type**, a kind of template. This turns out to be a surprisingly powerful concept when combined with multiple dispatch.

Let's try this out:

In [44]:
Vol(3)

Vol{Int64}(3)

In [45]:
Vol(3.5)

Vol{Float64}(3.5)

In [46]:
Vol("hello")

Vol{ASCIIString}("hello")

Here we find a new problem: the volume should be a real number. We can specify this restriction:

In [47]:
workspace()

In [49]:
Int <: Real

true

In [50]:
Float64 <: Real

true

In [51]:
BigFloat <: Real

true

In [53]:
Integer <: Real

true

In [55]:
workspace()

In [56]:
immutable Vol{T <: Real}  # only accept types T that are subtypes of Real
    value::T   
end

In [57]:
Vol("hello")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Vol{T<:Real}}, ::ASCIIString)
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
  Vol{T<:Real}(!Matched::T<:Real)
while loading In[57], in expression starting on line 1

In [58]:
Vol(3)

Vol{Int64}(3)

Let's put back the inner constructor:

In [62]:
workspace()

In [63]:
immutable Vol{T<:Real}
    value::T   
    
    function Vol(x)
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new(x)
    end
end

In [64]:
Vol(3)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Vol{T<:Real}}, ::Int64)
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[64], in expression starting on line 1

In [65]:
methods(Vol)

2-element Array{Any,1}:
 call{T}(::Type{T}, arg) at essentials.jl:56    
 call{T}(::Type{T}, args...) at essentials.jl:57

Suddenly we can no longer construct objects of our new type! This turns out to be due to the fact that defining an inner constructor prevents the usual default constructors from being defined. We will fix this in a minute by defining them ourselves.

We *can* still construct an object, but we have to explicitly specify the type:

In [66]:
Vol{Int}(3)

Vol{Int64}(3)

This is a pain and undesirable -- the point of Julia is that *it* is supposed to infer the types for us. 

The solution is to analyse what we want Julia to do, and then tell Julia to do exactly that. 

We would like Julia to look at the type of the object `x` passed to the constructor - call it `T` - and automatically use the constructor *for `Vol{T}`*, passing `x` as an argument, as we just did in the explicit call.

The syntax to do this looks strange at first (and second) glance:

In [68]:
f(x::T) = 2x

LoadError: LoadError: UndefVarError: T not defined
while loading In[68], in expression starting on line 1

In [70]:
Vol(x::T) = Vol{T}(x)

Vol{T<:Real}

Now it works:

In [71]:
Vol(3)

Vol{Int64}(3)

In [72]:
methods(Vol)

3-element Array{Any,1}:
 call{T}(::Type{Vol{T<:Real}}, x::T) at In[70]:1
 call{T}(::Type{T}, arg) at essentials.jl:56    
 call{T}(::Type{T}, args...) at essentials.jl:57

What is going on here? We seem to almost be repeating ourselves, writing the same both on the left and right of the assignment operator `=`. Once again, we are telling Julia to treat `T` as a type parameter. However, the meaning of `{T}` is *different* on the two sides.

- The left-hand side means: "for **any** type `T`, define a function `Vol` that takes a parameter `x` *of type `T`*".
This can be thought of as a specification for a matching algorithm: look at the function call and see if it matches this "template".


- The assignment operator has its usual meaning: "define the function on the left by the expression on the right".


- The right-hand side means: "call the constructor `Vol{T}` with argument `x`".

The point is that by the time the right-hand side is evaluated, Julia has **already realised** what `T` must be, since the function definition is matched by a call to `Vol` with an argument `x` of *any* type `T`!

The parametrised definition can be thought of as defining a "potentially infinite" number of different functions. Each of them is only instantiated, however, when the function is actually called with an argument of the corresponding type.

Furthermore, Julia is sophisticated enough that we can make multiple definitions like this. For example, we may want to store all volumes that are integers as exact integers, but all floating-point volumes as `BigFloat`s. We can do this by making a special method for floats:

In [73]:
Vol{T<:AbstractFloat}(x::T) = Vol{BigFloat}(x)

Vol{T<:Real}

In [74]:
Vol(3.5)

Vol{BigFloat}(3.500000000000000000000000000000000000000000000000000000000000000000000000000000)

Passing in integers does *not* match this new definition, and so still uses the previous definition:

In [75]:
Vol(3)

Vol{Int64}(3)

## Arithmetic

We would now like to define arithmetic operations:

In [76]:
V1 = Vol(3)
V2 = Vol(4)

Vol{Int64}(4)

In [77]:
V1 + V2

LoadError: LoadError: MethodError: `+` has no method matching +(::Vol{Int64}, ::Vol{Int64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
while loading In[77], in expression starting on line 1

Since we have not specified that `+` means for our type, Julia cannot know what to do.

We may, for some reason, want to define the sum of volumes only if they contain of the same type. We can follow the idea and syntax discussed for the constructor:

In [78]:
import Base.+

+{T}(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)

+ (generic function with 172 methods)

This looks a bit noisy, but we just have to parse it piece by piece: it is a method of `+`, that is parametric with one type parameter `T`, and whose arguments are `Vol`s parametrised by **the same** `T`. This is important, since it means that the definition **will not match** function calls with two different types. This is one aspect of **multiple dispatch**, i.e. the fact that Julia takes into account the types of *all* arguments to determine which method is called.

In [79]:
V1 + V2

Vol{Int64}(7)

However, this may be too restrictive; for example, we may wish to be able to add volumes with different integer types inside:

In [ ]:
Vol{Int64}(1) + Vol{BigInt}(3)

We can again define a more specialised method:

In [80]:
promote_type(Int32, Int64)

Int64

In [81]:
promote_type(Int32, BigInt)

BigInt

In [85]:
promote_type(Int, ASCIIString)

Any

In [87]:
+{T1<:Integer, T2<:Integer}(V1::Vol{T1}, V2::Vol{T2}) = Vol{promote_type(T1,T2)}(V1.value + V2.value)

    +(Main.Vol{#T1<:Integer}, Main.Vol{#T2<:Integer}) at In[87]:1
is ambiguous with: 
    +(Main.Vol{#T<:Any}, Main.Vol{#T<:Any}) at In[78]:3.
To fix, define 
    +(Main.Vol{_<:Integer}, Main.Vol{_<:Integer})
before the new definition.


+ (generic function with 173 methods)

[We could have left out the type parameter on the right-hand side, but this leads to a method ambiguity warning, in which two different methods match the combination of argument types, in this case when we have two equal integer types.]

Here, we have *two* type parameters that are both subtypes of the abstract `Integer` type, and `promote_type(T,S)` is a function from `Base` that determines the smallest supertype of both `T` and `S` that is able to correctly represent values of both types.

In [88]:
Vol{Int64}(1) + Vol{BigInt}(3)

Vol{BigInt}(4)

In [89]:
Vol{Float64}(1) + Vol{BigFloat}(3)

LoadError: LoadError: MethodError: `+` has no method matching +(::Vol{Float64}, ::Vol{BigFloat})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{T}(::Vol{T}, !Matched::Vol{T})
while loading In[89], in expression starting on line 1

Note that sometimes such a redefinition will apparently not work, since the previous definition, *even if it threw an error*, will be cached by Julia. In this case, it will be necessary to clear the workspace first and redefine everything. This can soon become tiresome and may be partially solved by moving the definitions out into a file, and `include`ing the file after calling `workspace()` to define the types. This kind of workflow discussion is best decided in practice.

## Adding more parameters 

Types may be parametrized by more than one parameter. In the case of volumes, we should not be able to "add" a 1-dimensional volume (length) to a 2-dimensional one (area), since this makes no sense physically or mathematically; on the other hand, we can multiply them to get a 3-dimensional volume. We thus need to add an integer parameter to the `Vol` type:

In [90]:
workspace()

In [91]:
immutable Vol{N,T<:Real}
    value::T   
end

Again, a default constructor is created, but now we must explicitly specify the types, since the type `N` cannot be inferred from the argument:

In [92]:
Vol{3, Float64}(3)

Vol{3,Float64}(3.0)

In [93]:
Vol{Int64, Float64}(3)

Vol{Int64,Float64}(3.0)

represents a three-dimensional volume of magnitude 3.

We wish to define the sum *only* when the dimensions are the same:

In [94]:
import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)

+ (generic function with 174 methods)

This will match only when both arguments have the same parameters `T` and `N`. We could also promote the types:

In [95]:
+{N,T1,T2}(V1::Vol{N,T1}, V2::Vol{N,T2}) = Vol{N,promote_type(T1,T2)}(V1.value + V2.value)

+ (generic function with 175 methods)

We make a useful outer constructor that can infer the type of its argument, but still needs an explicit dimension:

In [96]:
Vol{T}(N, x::T) = Vol{N,T}(x)

Vol{N,T<:Real}

In [97]:
Vol(3, 4.5)

Vol{3,Float64}(4.5)

We would also like to display objects in a nicer way:

In [99]:
import Base.show
show{N,T}(io::IO, x::Vol{N,T}) = print(io, N, "-dimensional volume with magnitude ", x.value)

show (generic function with 106 methods)

In [101]:
A = Vol(2, 3.0)   # volume of dimension 2 with value 3.0

2-dimensional volume with magnitude 3.0

In [102]:
ℓ = Vol(1, 10.0)  # write as \ell<TAB>

1-dimensional volume with magnitude 10.0

In [103]:
A + ℓ

LoadError: LoadError: MethodError: `+` has no method matching +(::Vol{2,Float64}, ::Vol{1,Float64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{N,T}(::Vol{N,T}, !Matched::Vol{N,T})
  +{N,T1,T2}(::Vol{N,T1}, !Matched::Vol{N,T2})
while loading In[103], in expression starting on line 1

It is not nice for the user to throw this slightly cryptic error message. We can provide a better error message when volumes of different dimension are added; we should check that volumes of the same dimension *still works*. Such things should be codified in unit test suites.

In [105]:
workspace()

In [106]:
immutable Vol{N,T<:Real}
    value::T   
end

Vol{T}(N, x::T) = Vol{N,T}(x)

import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)
+{N1,N2,T}(V1::Vol{N1,T}, V2::Vol{N2,T}) = throw(ArgumentError("Volumes of different dimension cannot be added"))

+ (generic function with 177 methods)

In [107]:
V1 = Vol(3, 3)
V2 = Vol(3, 4)
V1 + V2

Vol{3,Int64}(7)

In [108]:
Vol(3, 3) + Vol(2, 4)

LoadError: LoadError: ArgumentError: Volumes of different dimension cannot be added
while loading In[108], in expression starting on line 1

**Exercise**
Define the product (`*`) of any two objects of type `Vol`.

## Conversion

We may want to define functions like `sin` that act on objects of our new type.
[We may *not* want to define such functions, on the other hand, since we could argue that a `Vol` is not a real number, but rather is a combination of a real number together with a dimension.]
Let's try it:

In [109]:
V1

Vol{3,Int64}(3)

In [110]:
sin(V1)

LoadError: LoadError: MethodError: `sin` has no method matching sin(::Vol{3,Int64})
while loading In[110], in expression starting on line 1

Of course, Julia doesn't know how to do this, since we haven't defined it.
However, an object of type `Vol` is just a simple wrapper around a number, so we could just say

In [111]:
import Base.sin
sin(x::Vol) = sin(x.value)

sin (generic function with 12 methods)

In [112]:
sin(V1)

0.1411200080598672

It is painful / boring / prone to error to do this for every function (although we will see in the Metaprogramming notebook how to do streamline this). An alternative is to tell Julia explicitly that a `Vol` is actually a kind of real number, i.e. a subtype of `Real`. [It is sometimes the case that this is mathematically not true, but it is a convenience to simplify the code, for example for interval subsets of the real line.]

In [116]:
workspace()

In [117]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [118]:
V = Vol{3,Float64}(3)

Vol{3,Float64}(3.0)

In [119]:
sin(V)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{AbstractFloat}, ::Vol{3,Float64})
This may have arisen from a call to the constructor AbstractFloat(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T<:AbstractFloat}(::Type{T<:AbstractFloat}, ::Real, !Matched::RoundingMode{T})
  call{T}(::Type{T}, ::Any)
  convert(::Type{AbstractFloat}, !Matched::Bool)
  ...
while loading In[119], in expression starting on line 1

We now get a different error. We can see which method Julia is now using:

In [120]:
@which sin(V)

sin(x::Real) at math.jl:139

This is a generic "fallback" ("catch-all") method, which is called when no more specialised method has been defined.

Following the link to the Julia source code, which is written in Julia and therefore amazingly understandable (with some work), and if we understand some metaprogramming, we find that the definition of `sin(x::Real)` is

    sin(x::Real) = sin(float(x))

We could just define `float` for our type, which is supposed to convert an object to it's most natural floating-point equivalent:

In [121]:
workspace()

In [122]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

import Base.float
float(V::Vol) = float(V.value)

float (generic function with 18 methods)

In [123]:
V = Vol{3,Float64}(3)
sin(V)

0.1411200080598672

However, in fact `float` is already defined for *any* `x::Real`:

In [124]:
workspace()

In [125]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [126]:
V = Vol{3,Float64}(3)

Vol{3,Float64}(3.0)

In [127]:
@which float(V)

float(x) at float.jl:121

In [128]:
float(3)

3.0

The definition reads:

    float(x) = convert(AbstractFloat, x)

Finally, we see that rather we need to investigate Julia's capabilities for [conversion and promotion](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/) and define how to `convert` an object of our type into a float:

In [129]:
import Base.convert
convert(::Type{AbstractFloat}, x::Vol) = float(x.value)

convert (generic function with 538 methods)

In [130]:
convert(AbstractFloat, V)

3.0

In [131]:
methods(convert)

# 538 methods for generic function "convert":
convert(::Type{Tuple{}}, ::Tuple{}) at essentials.jl:61
convert{T<:Tuple{Any,Vararg{Any}}}(::Type{T<:Tuple{Any,Vararg{Any}}}, x::T<:Tuple{Any,Vararg{Any}}) at essentials.jl:98
convert{T<:Tuple{Any,Vararg{Any}}}(::Type{T<:Tuple{Any,Vararg{Any}}}, x::Tuple{Any,Vararg{Any}}) at essentials.jl:96
convert(::Type{Tuple}, x::Tuple) at essentials.jl:62
convert{T}(::Type{Tuple{Vararg{T}}}, x::Tuple) at essentials.jl:63
convert{T<:Union{Int8,UInt8}}(::Type{Cstring}, p::Ptr{T<:Union{Int8,UInt8}}) at c.jl:61
convert(::Type{Cwstring}, p::Ptr{Int32}) at c.jl:62
convert{T<:Union{Int8,UInt8}}(::Type{Ptr{T<:Union{Int8,UInt8}}}, p::Cstring) at c.jl:63
convert(::Type{Ptr{Int32}}, p::Cwstring) at c.jl:64
convert(::Type{Cstring}, s::Symbol) at c.jl:83
convert{T}(::Type{UnitRange{T}}, r::UnitRange{T}) at range.jl:587
convert{T}(::Type{UnitRange{T}}, r::UnitRange{T<:Real}) at range.jl:588
convert{T1,T2}(::Type{StepRange{T1,T2}}, r::StepRange{T1,T2}) at range.jl:592
convert{T}(::Type{StepRange{T,S}}, r::UnitRange{T}) at range.jl:598
convert{T1,T2}(::Type{StepRange{T1,T2}}, r::Range{T}) at range.jl:596
convert{T}(::Type{FloatRange{T}}, r::FloatRange{T}) at range.jl:603
convert{T}(::Type{FloatRange{T}}, r::FloatRange{T<:AbstractFloat}) at range.jl:604
convert{T}(::Type{FloatRange{T<:AbstractFloat}}, r::OrdinalRange{T,S}) at range.jl:611
convert{T}(::Type{FloatRange{T}}, r::OrdinalRange{T,S}) at range.jl:609
convert{T}(::Type{LinSpace{T}}, r::LinSpace{T}) at range.jl:616
convert{T}(::Type{LinSpace{T}}, r::LinSpace{T<:AbstractFloat}) at range.jl:617
convert{T}(::Type{LinSpace{T<:AbstractFloat}}, r::OrdinalRange{T,S}) at range.jl:624
convert{T}(::Type{LinSpace{T}}, r::OrdinalRange{T,S}) at range.jl:622
convert{T}(::Type{LinSpace{T}}, r::FloatRange{T<:AbstractFloat}) at range.jl:630
convert{T}(::Type{LinSpace{T<:AbstractFloat}}, r::FloatRange{T}) at range.jl:632
convert{T}(::Type{Array{T,1}}, r::Range{T}) at range.jl:671
convert(::Type{Bool}, x::Bool) at bool.jl:5
convert(::Type{Bool}, x::Rational{T<:Integer}) at rational.jl:75
convert(::Type{Bool}, x::Float16) at float16.jl:101
convert(::Type{Bool}, x::BigFloat) at mpfr.jl:164
convert(::Type{Bool}, x::Real) at bool.jl:6
convert(::Type{Int8}, x::UInt8) at int.jl:193
convert(::Type{Int8}, x::Int16) at int.jl:172
convert(::Type{Int8}, x::UInt16) at int.jl:170
convert(::Type{Int8}, x::Int32) at int.jl:172
convert(::Type{Int8}, x::UInt32) at int.jl:170
convert(::Type{Int8}, x::Int64) at int.jl:172
convert(::Type{Int8}, x::UInt64) at int.jl:170
convert(::Type{Int8}, x::Int128) at int.jl:172
convert(::Type{Int8}, x::UInt128) at int.jl:170
convert(::Type{Int8}, x::Bool) at int.jl:187
convert(::Type{UInt8}, x::Int8) at int.jl:193
convert(::Type{UInt8}, x::Int16) at int.jl:175
convert(::Type{UInt8}, x::UInt16) at int.jl:175
convert(::Type{UInt8}, x::Int32) at int.jl:175
convert(::Type{UInt8}, x::UInt32) at int.jl:175
convert(::Type{UInt8}, x::Int64) at int.jl:175
convert(::Type{UInt8}, x::UInt64) at int.jl:175
convert(::Type{UInt8}, x::Int128) at int.jl:175
convert(::Type{UInt8}, x::UInt128) at int.jl:175
convert(::Type{UInt8}, x::Bool) at int.jl:187
convert(::Type{Int16}, x::Int8) at int.jl:183
convert(::Type{Int16}, x::UInt8) at int.jl:187
convert(::Type{Int16}, x::UInt16) at int.jl:193
convert(::Type{Int16}, x::Int32) at int.jl:172
convert(::Type{Int16}, x::UInt32) at int.jl:170
convert(::Type{Int16}, x::Int64) at int.jl:172
convert(::Type{Int16}, x::UInt64) at int.jl:170
convert(::Type{Int16}, x::Int128) at int.jl:172
convert(::Type{Int16}, x::UInt128) at int.jl:170
convert(::Type{Int16}, x::Bool) at int.jl:187
convert(::Type{UInt16}, x::Int8) at int.jl:181
convert(::Type{UInt16}, x::UInt8) at int.jl:187
convert(::Type{UInt16}, x::Int16) at int.jl:193
convert(::Type{UInt16}, x::Int32) at int.jl:175
convert(::Type{UInt16}, x::UInt32) at int.jl:175
convert(::Type{UInt16}, x::Int64) at int.jl:175
convert(::Type{UInt16}, x::UInt64) at int.jl:175
convert(::Type{UInt16}, x

Now any standard unary function works:

In [132]:
sin(V), cos(V), exp(V)

(0.1411200080598672,-0.9899924966004454,20.085536923187668)

`::Type{AbstractFloat}` refers to an argument which has type "type of `AbstractFloat`"; the only object that has that type is `AbstractFloat` itself; this thus matches function calls of the form 

    convert(AbstractFloat, V)

In [ ]:
convert(AbstractFloat, V2)

## Promotion 

In the case of `Vol`, it is not natural to define all arithmetic operations, so let's simplify even further and just wrap a `Float64` in a `Prob` type, representing a probability. [We will assume that it makes sense to define all arithmetic operations here, although in a given application that may not be the case.]

In [ ]:
workspace()

In [134]:
immutable Prob{T<:AbstractFloat} <: Real
    _::T
    
    function Prob(x)
        (!(0 <= x <= 1)) && throw(ArgumentError("Probability must be between 0 and 1."))
        new(x)
    end
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

Prob{T<:AbstractFloat}

[Note that `_` is a valid variable name in Julia, and is often used for a variable whose name we don't care about.]

In [135]:
x = Prob(0.5)

Prob{Float64}(0.5)

We see that multiplying `x` by a real number does not give a `MethodError`:

In [136]:
3x

LoadError: LoadError: no promotion exists for Int64 and Prob{Float64}
while loading In[136], in expression starting on line 1

since a `Vol` is a subtype of `Number`. 

Rather, Julia is trying to use its promotion machinery, in which such operations are handled by **promoting** both arguments to the same type:

In [ ]:
@which 3x

We can see the process using `@which` (or using the debugger in Julia v0.5):

In [ ]:
@which promote(3, x)

In [ ]:
@which promote_type(Int, Prob{Float64})

which is defined in terms of `promote_rule(T,S)`, e.g.

In [ ]:
@which promote_rule(Float64, Int)

So if we want to have promotion work, we need to define `promote_rule`:

In [142]:
workspace()

In [143]:
immutable Prob{T<:AbstractFloat} <: Real
    value::T
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

import Base.*
*{T<:AbstractFloat}(x::Prob{T}, y::Prob{T}) = Prob{T}(x.value*y.value)

* (generic function with 140 methods)

In [144]:
p1 = Prob(0.3)
p2 = Prob(0.4)
p1 * p2

Prob{Float64}(0.12)

In [145]:
0.3 * p2

LoadError: LoadError: no promotion exists for Float64 and Prob{Float64}
while loading In[145], in expression starting on line 1

In [147]:
import Base: promote_rule, convert

promote_rule{S<:Real, T<:Real}(::Type{Prob{T}}, ::Type{S}) = Prob{promote_type(S, T)}

convert{S<:Real, T<:Real}(::Type{Prob{T}}, x::S) = Prob(float(x))
convert{T<:Real}(::Type{AbstractFloat}, x::Prob{T}) = x.value


convert (generic function with 540 methods)

In [148]:
x = Prob(0.5)


Prob{Float64}(0.5)

In [149]:
3x

Prob{Float64}(1.5)

In [ ]:
promote(3, x)

These rules are always tricky to get right!

## `Vector`s of objects of parametric type

Suppose we make a vector of rationals, which is a parametric type

In [150]:
v = [3//4, 4//5]

2-element Array{Rational{Int64},1}:
 3//4
 4//5

In [151]:
typeof(v)

Array{Rational{Int64},1}

A **typealias**, that is, an alternative (shorter and/or more intuitive) name for the type `Array{T,1}` is `Vector{T}`:

In [152]:
Array{Rational{Int64},1} === Vector{Rational{Int}}  

true

Here, the `===` operator checks for identity (rather than just equality) of objects:

In [153]:
1 == 1.0

true

In [154]:
1 === 1.0

false

We also have a subtype relationship:

In [155]:
Rational{Int} <: Rational

true

where `Rational` is an abstract type, meaning that we cannot create objects of type `Rational`; they must always be parametrised.

**However**, we have

In [156]:
Vector{Rational{Int}} <: Vector{Rational}

false

Thus if we define a method

In [157]:
h(x::Vector{Rational}) = 2x

h (generic function with 1 method)

that accepts arguments that are `Vector`s of `Rational`s, we have

In [158]:
v = [3//4, 4//5]
h(v)

LoadError: LoadError: MethodError: `h` has no method matching h(::Array{Rational{Int64},1})
while loading In[158], in expression starting on line 2

i.e. `h` **does not** accept arguments of type `Vector{Rational{Int}}`. Instead, we must parametrize explicitly:

In [159]:
h{T}(x::Vector{Rational{T}}) = 2x

h (generic function with 2 methods)

In [160]:
h(v)

2-element Array{Rational{Int64},1}:
 3//2
 8//5

Here, the syntax `{T}` means "define a function `h` with this template, for *any* type `T`.